In [1]:
#Step 1 - Import the Library
import psycopg2
import pandas as pd

In [2]:
def create_database():
   # connect to default database
   # we used conn = psycopg2.connect("host=hostname dbname=database name user=username password=database password")
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=Swaps123@")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE accounts")
    cur.execute("CREATE DATABASE accounts")
    
    # close the connection to default database
    conn.close()
    
    # connect to sparkify database
    conn = psycopg2.connect("host=localhost dbname=accounts user=postgres password=Swaps123@")
    cur = conn.cursor()
    
    return cur, conn

In [3]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [4]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [5]:
accountscountry = pd.read_csv("C:/Users/sc130/Downloads/3_water_withdrawal_factor_by_fuel.csv")

In [6]:
accountscountry.head()

,Country,Biogas,Biomass,Coal Gases,Geothermal,Hard Coal,Heavy Fuel Oil,Hydro,Lignite,Natural Gas,Nuclear,Peat,Photovoltaics,Solar Thermal,Waste,Wind
0,Argentina,0,0,0,0,0,30,1114,0,19,39,0,0,0,0,0
1,Australia,2,3,3,0,3,3,520,5,3,0,0,0,0,0,0
2,Austria,23,13,21,0,13,9,7100,0,3,0,0,0,0,23,0
3,Belgium,5,14,15,0,17,6,2613,0,5,30,0,0,0,23,0
4,Brazil,27,17,23,0,23,24,348,36,20,39,0,0,0,0,0


In [7]:
accountscountry_clean = accountscountry[['Country', 'Biogas', 'Biomass', 'Geothermal']]

In [8]:
accountscountry_clean.head()

,Country,Biogas,Biomass,Geothermal
0,Argentina,0,0,0
1,Australia,2,3,0
2,Austria,23,13,0
3,Belgium,5,14,0
4,Brazil,27,17,0


In [9]:
accountsdata = pd.read_csv("C:/Users/sc130/Downloads/4_water_consumption_factor_by_fuel.csv")

In [10]:
accountsdata.columns

Index(['Country', 'Biogas', 'Biomass', 'Coal Gases', 'Geothermal', 'Hard Coal',
       'Heavy Fuel Oil', 'Hydro', 'Lignite', 'Natural Gas', 'Nuclear', 'Peat',
       'Photovoltaics', 'Solar Thermal', 'Waste', 'Wind'],
      dtype='object')

In [11]:
accountsdata_clean = accountsdata[['Country', 'Hydro', 'Nuclear']]

In [12]:
accountsdata_clean.head()

,Country,Hydro,Nuclear
0,Argentina,8.22,0.45
1,Australia,9.62,0.00
2,Austria,1.02,0.00
3,Belgium,3.15,0.50
4,Brazil,7.22,0.45


In [13]:
accountseries = pd.read_csv("C:/Users/sc130/Downloads/6_percentage_by_electricity_source.csv")

In [14]:
accountseries.head()

,Country,Biogas,Biomass,Coal Gases,Geothermal,Hard Coal,Heavy Fuel Oil,Hydro,Lignite,Natural Gas,Nuclear,Peat,Photovoltaic,Solar Thermal,Waste,Wind
0,Argentina,0.0,2.0,0.3,0.0,2.5,13.8,29.2,0.0,47.6,4.1,0.0,0.0,0.0,0.0,0.5
1,Australia,0.7,0.8,0.0,0.0,42.6,2.0,7.4,18.6,21.9,0.0,0.0,2.0,0.0,0.0,4.1
2,Austria,0.9,5.3,3.0,0.0,4.5,0.9,68.5,0.0,8.3,0.0,0.0,1.2,0.0,1.5,5.9
3,Belgium,1.3,3.6,3.0,0.0,3.1,0.3,2.1,0.0,26.7,46.6,0.0,4.0,0.0,2.9,6.4
4,Brazil,0.1,7.7,1.4,0.0,1.8,6.0,63.3,1.3,13.7,2.6,0.0,0.0,0.0,0.0,2.1


In [15]:
accountseries_clean = accountseries[['Nuclear', 'Photovoltaic', 'Wind']]

In [16]:
accountseries_clean.head()

,Nuclear,Photovoltaic,Wind
0,4.1,0.0,0.5
1,0.0,2.0,4.1
2,0.0,1.2,5.9
3,46.6,4.0,6.4
4,2.6,0.0,2.1


In [17]:
cur, conn = create_database()

In [18]:
songplay_table_create = ("""CREATE TABLE IF NOT EXISTS accountscountry(
Country VARCHAR PRIMARY KEY,
Biogas numeric,
Biomass numeric,
Geothermal numeric
)""")

In [19]:
cur.execute(songplay_table_create)
conn.commit()

In [20]:
accounts_data_table_create = ("""CREATE TABLE IF NOT EXISTS accountsdata(
Country VARCHAR,
Hydro numeric,
Nuclear numeric
)""")

In [21]:
cur.execute(accounts_data_table_create)
conn.commit()

In [22]:
accounts_series_table_create = ("""CREATE TABLE IF NOT EXISTS accountseries(
Nuclear numeric,
Photovoltaic numeric,
Wind numeric
)""")

In [23]:
cur.execute(accounts_series_table_create)
conn.commit()

In [24]:
accounts_country_create_table_insert = ("""INSERT INTO accountscountry(
Country,
Biogas,
Biomass,
Geothermal)
VALUES (%s, %s, %s, %s)
""")

In [25]:
for i, row in accountscountry_clean.iterrows():
    cur.execute(accounts_country_create_table_insert, list(row))

In [26]:
conn.commit()

In [27]:
accounts_data_table_insert = ("""INSERT INTO accountsdata(
Country,
Hydro,
Nuclear)
VALUES (%s, %s, %s)
""")

In [28]:
for i, row in accountsdata_clean.iterrows():
    cur.execute(accounts_data_table_insert, list(row))

In [29]:
conn.commit()

In [30]:
accounts_series_table_insert = ("""INSERT INTO accountseries(
Nuclear,
Photovoltaic,
Wind)
VALUES (%s, %s, %s)
""")

In [31]:
for i, row in accountseries_clean.iterrows():
    cur.execute(accounts_series_table_insert, list(row))

In [32]:
conn.commit()